## 04_gold_views
### Purpose: build Gold layer for Analytics and BI consumption
Inputs: *__clean_

Outputs: *__gold_

### Wide fact (with dims)

In [0]:
CREATE OR REPLACE VIEW curso.ventas.ventas_gold AS
SELECT
  -- Fact
  f.num_factura,
  f.fecha_venta,
  f.id_cliente,
  f.id_producto,
  f.id_vendedor,
  f.cantidad,

  -- Products
  p.nombre AS producto_nombre,
  p.familia AS producto_familia,
  p.precio_unitario AS producto_precio_unitario,
  (f.cantidad * p.precio_unitario) AS importe_total,

  -- Clients
  c.nombre AS cliente_nombre,
  c.apellido AS cliente_apellido,
  c.sexo AS cliente_sexo,
  c.fecha_nacimiento AS cliente_fecha_nacimiento,

  -- Employees (+ flag to identify valid/invalid employees)
  e.nombre AS empleado_nombre,
  e.apellido AS empleado_apellido,
  CONCAT(e.nombre, ' ', e.apellido) AS empleado_nombre_completo,
  e.sucursal AS id_sucursal,

  CASE WHEN e.id_vendedor IS NULL THEN 0 ELSE 1 END AS flag_empleado_valido,

  -- Stores (through Employees table)
  l.nombre AS local_nombre,
  l.tipo AS local_tipo

FROM curso.ventas.facturas_clean f

LEFT JOIN curso.ventas.productos_clean p
  ON f.id_producto = p.id_producto

LEFT JOIN curso.ventas.clientes_clean c
  ON f.id_cliente = c.id_cliente

LEFT JOIN curso.ventas.empleados_clean e
  ON f.id_vendedor = e.id_vendedor

LEFT JOIN curso.ventas.locales_clean l
  ON e.sucursal = l.id_sucursal;


In [0]:
SELECT *
FROM curso.ventas.ventas_gold
LIMIT 10;

num_factura,fecha_venta,id_cliente,id_producto,id_vendedor,cantidad,producto_nombre,producto_familia,producto_precio_unitario,importe_total,cliente_nombre,cliente_apellido,cliente_sexo,cliente_fecha_nacimiento,empleado_nombre,empleado_apellido,empleado_nombre_completo,id_sucursal,flag_empleado_valido,local_nombre,local_tipo
0000-05444682,2022-11-13,1668,86,38,56,Dreyer,Helados,156,8736,Fortunata,Alarcón,null,1953-01-02,Ramon,Gómez,Ramon Gómez,10,1,Éxito La Ceja,Comercio de cercania
0000-11294112,2022-08-10,2447,90,63,84,Solís,Culinarios,102,8568,Modesta,Moya,M,1988-06-20,null,null,null,null,0,null,null
0000-13526698,2021-08-22,1109,106,33,66,Sahne Nuss,Chocolates,205,13530,Marcelino,Vilalta,O,1952-03-19,Carlos,González,Carlos González,9,1,Éxito Indiana Mall,Comercio de cercania
0000-32356836,2022-10-13,1509,115,21,156,Purina Dog Chow,Comidas para animales,226,35256,Ambrosio,Catalá,O,1970-10-16,Ramon,Pérez,Ramon Pérez,10,1,Éxito La Ceja,Comercio de cercania
0000-52191864,2022-06-20,1042,112,15,155,Super 8,Chocolates,68,10540,Isaac,Cid,M,1957-06-08,Jorge,Sánchez,Jorge Sánchez,14,1,Éxito La 33,Supermercado
0000-60937597,2022-03-25,1016,109,24,198,Baby ruth,Chocolates,240,47520,Fanny,Seco,F,1973-06-08,Jorge,Pérez,Jorge Pérez,13,1,Éxito Junin,Supermercado
0000-63017602,2023-08-14,1882,60,14,42,Bonka,Café,63,2646,Ulises,Muñoz,M,1972-04-26,Carlos,Gómez,Carlos Gómez,14,1,Éxito La 33,Supermercado
0000-83897840,2022-03-28,468,88,58,41,Maggi,Culinarios,162,6642,Pío,Sáez,F,1962-03-09,null,null,null,null,0,null,null
0000-85450647,2022-08-09,181,14,33,114,Nutrición y Salud,Leche,154,17556,Otilia,Cabo,null,1992-01-23,Carlos,González,Carlos González,9,1,Éxito Indiana Mall,Comercio de cercania
0000-86250165,2023-02-16,306,38,11,29,Gold Flakes,Cereales,34,986,Anastasio,Mariño,O,2001-08-13,Maria,Martínez,Maria Martínez,5,1,Éxito Del Este,Vecino


### Daily sales

In [0]:
CREATE OR REPLACE VIEW curso.ventas.ventas_diarias_gold AS
SELECT 
  fecha_venta, 
  COUNT(DISTINCT num_factura) AS facturas,
  SUM(cantidad) AS unidades,
  SUM(importe_total) AS importe_total
FROM curso.ventas.ventas_gold
GROUP BY fecha_venta;

In [0]:
SELECT *
FROM curso.ventas.ventas_diarias_gold
LIMIT 10;

fecha_venta,facturas,unidades,importe_total
2022-11-13,92,10163,1309335
2022-08-10,79,7677,1140512
2021-08-22,90,9101,1309671
2022-10-13,83,7745,1142694
2022-06-20,97,9245,1222831
2022-03-25,93,9626,1375850
2023-08-14,98,9580,1314358
2022-03-28,79,7103,1080436
2022-08-09,92,9239,1218886
2023-02-16,97,9051,1182102


### Sales by product

In [0]:
CREATE OR REPLACE VIEW curso.ventas.ventas_por_producto_gold AS
SELECT
  id_producto,
  producto_nombre,
  producto_familia,
  SUM(cantidad) AS unidades,
  SUM(importe_total) AS importe_total,
  COUNT(DISTINCT num_factura) AS facturas
FROM curso.ventas.ventas_gold
GROUP BY id_producto, producto_nombre, producto_familia;


### Sales by store

In [0]:
SELECT *
FROM curso.ventas.ventas_por_producto_gold
LIMIT 10;

id_producto,producto_nombre,producto_familia,unidades,importe_total,facturas
86,Dreyer,Helados,84070,13114920,842
90,Solís,Culinarios,86364,8809128,849
106,Sahne Nuss,Chocolates,89032,18251560,860
115,Purina Dog Chow,Comidas para animales,83343,18835518,846
112,Super 8,Chocolates,84643,5755724,829
109,Baby ruth,Chocolates,89610,21506400,880
60,Bonka,Café,84497,5323311,825
88,Maggi,Culinarios,77911,12621582,783
14,Nutrición y Salud,Leche,85396,13150984,839
38,Gold Flakes,Cereales,82631,2809454,815


In [0]:
CREATE OR REPLACE VIEW curso.ventas.ventas_por_sucursal_gold AS
SELECT
  id_sucursal,
  local_nombre,
  local_tipo,
  SUM(cantidad) AS unidades,
  SUM(importe_total) AS importe_total,
  COUNT(DISTINCT num_factura) AS facturas,
  SUM(CASE WHEN flag_empleado_valido = 0 THEN 1 ELSE 0 END) AS facturas_empleado_invalido
FROM curso.ventas.ventas_gold
GROUP BY
  id_sucursal,
  local_nombre,
  local_tipo;

In [0]:
SELECT * 
FROM curso.ventas.ventas_por_sucursal_gold
LIMIT 10;

id_sucursal,local_nombre,local_tipo,unidades,importe_total,facturas,facturas_empleado_invalido
10,Éxito La Ceja,Comercio de cercania,623509,87719090,6104,0
null,null,null,1392716,192130608,13922,13922
9,Éxito Indiana Mall,Comercio de cercania,618328,86263728,6119,0
14,Éxito La 33,Supermercado,1230917,171107614,12266,0
13,Éxito Junin,Supermercado,753314,104728345,7574,0
5,Éxito Del Este,Vecino,468066,64887456,4607,0
4,Éxito Colombia,Hipermercado,455870,63370336,4531,0
8,Éxito Gran Vía,Supermercado,615845,86079524,6174,0
1,Éxito Aranjuez,Supermercado,940322,130839954,9325,0
12,Éxito Itagüí,Supermercado,938668,130984032,9347,0


### Sales by employee

In [0]:
CREATE OR REPLACE VIEW curso.ventas.ventas_por_empleado_gold AS
SELECT
  id_vendedor,
  empleado_nombre_completo,
  flag_empleado_valido,
  SUM(cantidad) AS unidades,
  SUM(importe_total) AS importe_total,
  COUNT(DISTINCT num_factura) AS facturas
FROM curso.ventas.ventas_gold
GROUP BY id_vendedor, empleado_nombre_completo, flag_empleado_valido;


In [0]:
SELECT *
FROM curso.ventas.ventas_por_empleado_gold
LIMIT 10;

id_vendedor,empleado_nombre_completo,flag_empleado_valido,unidades,importe_total,facturas
38,Ramon Gómez,1,152712,21037602,1468
63,null,0,150478,21012390,1495
33,Carlos González,1,154271,21372876,1494
21,Ramon Pérez,1,154242,21718850,1516
15,Jorge Sánchez,1,152950,20785730,1505
24,Jorge Pérez,1,154367,21581394,1531
14,Carlos Gómez,1,160388,22690516,1603
58,null,0,149075,21004169,1473
11,Maria Martínez,1,161652,22892287,1603
13,Jose Álvarez,1,152573,21271650,1515
